# Exercise 9 - Solution

### Task

Implement a multilayer Fuly Connected Network from scratch using Numpy. Use the class structure provided below. Four member functions are to be implemented.
* 'forward(self, x)' for the forward prediction
* 'backward(self, y)' for the backpropagation in order to compute the gradients
* 'zero_grad(self)' to reset the gradients to zero (to be used before a second prediction/backpropagation)
* 'step(self, lr)' to update the weights and biases with gradient descent

The most challenging task is to compute the gradients. Therefore split the implementation up into two steps:

* implement the gradient computation for a sample size of one
* implement the gradient computation for an arbitrary sample size (use the currentWeightGradients and currentBiasGradients to store the intermediate results)

To check the results, you can verify the gradients by comparing them to PyTorch. A code copying the custom neural network PyTorch, where the automatic differentiation is provided below.

After a sccessful verification, use the neural network to learn a function. The training algorithm is provided below. Using the Adam optimizer implementation from exercise 2.3, the training can be improved. It is almost impossible to learn a sufficiently complex function with only gradient descent.

Finally, compare the implementation to a PyTorch implementation.

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

neural network class

In [2]:
class neuralNetwork:
    def __init__(
            self, layers, activation, activationGradient, xavierInitialization=True
    ):
        self.L = len(layers)
        if xavierInitialization == True:
            self.weights = [
                torch.nn.init.xavier_uniform_(
                    torch.zeros((layers[i], layers[i + 1])),
                    gain=torch.nn.init.calculate_gain("sigmoid"),
                ).numpy()
                for i in range(self.L - 1)
            ]
        else:
            self.weights = [
                np.random.rand(layers[i], layers[i + 1]) for i in range(self.L - 1)
            ]
        self.biases = [np.random.rand(1, layers[i + 1]) for i in range(self.L - 1)]

        self.layerActivations = []

        self.weightGradients = [
            np.zeros((layers[i], layers[i + 1])) for i in range(self.L - 1)
        ]
        self.biasGradients = [np.zeros((1, layers[i + 1])) for i in range(self.L - 1)]

        self.currentWeightGradients = [
            np.zeros((layers[i], layers[i + 1])) for i in range(self.L - 1)
        ]
        self.currentBiasGradients = [
            np.zeros((1, layers[i + 1])) for i in range(self.L - 1)
        ]

        self.activation = activation
        self.activationGradient = activationGradient

    def forward(self, x):
        self.layerActivations = []
        a = x
        self.layerActivations.append(a)
        for i in range(self.L - 1):
            z = a @ self.weights[i] + self.biases[i]
            self.layerActivations.append(z)
            a = self.activation(z)
        return a

    def backward(self, y):
        if len(self.layerActivations) > 0:
            numberOfSamples = len(self.layerActivations[0])

            if numberOfSamples == 1:
                deltaL = -(
                        y - self.activation(self.layerActivations[self.L - 1])
                ) * self.activationGradient(self.layerActivations[self.L - 1])
                self.biasGradients[self.L - 2] = deltaL
                for i in range(
                        self.L - 2
                ):
                    deltal = np.sum(
                        self.weights[self.L - 2 - i]
                        * self.biasGradients[self.L - 2 - i],
                        1,
                    ) * self.activationGradient(self.layerActivations[self.L - 2 - i])
                    self.biasGradients[self.L - 3 - i] = deltal

                self.weightGradients[0] = (
                        np.transpose(self.layerActivations[0]) @ self.biasGradients[0]
                )
                for i in range(1, self.L - 1):
                    self.weightGradients[i] = (
                            np.transpose(self.activation(self.layerActivations[i]))
                            @ self.biasGradients[i]
                    )

            elif numberOfSamples > 1:
                for j in range(numberOfSamples):

                    deltaL = -(
                            y[j: j + 1]
                            - self.activation(self.layerActivations[self.L - 1][j: j + 1])
                    ) * self.activationGradient(
                        self.layerActivations[self.L - 1][j: j + 1]
                    )
                    self.currentBiasGradients[self.L - 2] = (
                            deltaL / numberOfSamples
                    )
                    self.biasGradients[self.L - 2] += self.currentBiasGradients[
                        self.L - 2
                        ]
                    for i in range(
                            self.L - 2
                    ):
                        deltal = np.sum(
                            self.weights[self.L - 2 - i]
                            * self.currentBiasGradients[self.L - 2 - i],
                            1,
                        ) * self.activationGradient(
                            self.layerActivations[self.L - 2 - i][j: j + 1]
                        )
                        self.currentBiasGradients[self.L - 3 - i] = deltal
                        self.biasGradients[self.L - 3 - i] += self.currentBiasGradients[
                            self.L - 3 - i
                            ]

                    self.currentWeightGradients[0] = (
                            np.transpose(self.layerActivations[0][j: j + 1])
                            @ self.currentBiasGradients[0]
                    )
                    self.weightGradients[0] += self.currentWeightGradients[0]
                    for i in range(1, self.L - 1):
                        self.currentWeightGradients[i] = (
                                np.transpose(
                                    self.activation(self.layerActivations[i][j: j + 1])
                                )
                                @ self.currentBiasGradients[i]
                        )
                        self.weightGradients[i] += self.currentWeightGradients[i]

        else:
            print("backward propagation not possible")

    def zero_grad(self):
        self.weightGradients = [
            np.zeros((layers[i], layers[i + 1])) for i in range(self.L - 1)
        ]
        self.biasGradients = [np.zeros((1, layers[i + 1])) for i in range(self.L - 1)]

    def step(self, lr):
        for i in range(self.L - 1):
            self.weights[i] -= lr * self.weightGradients[i]
            self.biases[i] -= lr * self.biasGradients[i]

model definition

In [8]:
layers = [1, 4, 4, 1]
sigmoid = lambda x: 1 / (1 + np.exp(-x))
sigmoidGradient = lambda x: sigmoid(x) * (1 - sigmoid(x))

model = neuralNetwork(layers, sigmoid, sigmoidGradient)

x = np.expand_dims(np.linspace(0, 1, 2), 1) + 0.2
y = np.sin(4 * np.pi * x) ** 2

prediction, cost computation & gradient computation

In [9]:
yPred = model.forward(x)

C = 0.5 * np.mean((yPred - y) ** 2)

model.backward(y)

## Verification with PyTorch

model definition and cloning of model parameters

In [10]:
class neuralNetworkTorch(torch.nn.Module):
    def __init__(self, layers, activationFunction=torch.nn.Sigmoid()):
        super().__init__()
        modules = []
        for i in range(len(layers) - 1):
            modules.append(torch.nn.Linear(layers[i], layers[i + 1]))
            modules.append(activationFunction)

        self.model = torch.nn.Sequential(*modules)

    def forward(self, x):
        return self.model(x)


modelTorch = neuralNetworkTorch(layers)

with torch.no_grad():
    for i, param in enumerate(modelTorch.parameters()):
        if i % 2 == 0:
            param.data = torch.from_numpy(model.weights[i // 2]).to(torch.float64).t()
        else:
            param.data = torch.from_numpy(model.biases[i // 2]).to(torch.float64)

prediction, cost computation & gradient computation

In [11]:
xTorch = torch.from_numpy(x).to(torch.float64)
yPredTorch = modelTorch.forward(xTorch)

CTorch = 0.5 * torch.mean((yPredTorch - torch.from_numpy(y).to(torch.float64)) ** 2)
CTorch.backward()

gradient comparison

In [12]:
layer = 0
print("weight:")
print(np.transpose(model.weightGradients[layer]))
print(list(modelTorch.parameters())[2 * layer].grad)

print("bias:")
print(model.biasGradients[layer])
print(list(modelTorch.parameters())[2 * layer + 1].grad)

weight:
[[-8.20482858e-06]
 [-1.11781064e-04]
 [ 2.74173667e-04]
 [ 2.01863001e-04]]
tensor([[-8.2048e-06],
        [-1.1178e-04],
        [ 2.7417e-04],
        [ 2.0186e-04]], dtype=torch.float64)
bias:
[[-1.12428905e-05 -1.95147987e-04  4.34993078e-04  2.75672713e-04]]
tensor([[-1.1243e-05, -1.9515e-04,  4.3499e-04,  2.7567e-04]],
       dtype=torch.float64)
